# Adaboost

In [14]:
import pickle

X_train, X_test, y_train, y_test = pickle.load(open("../experiments/higgs/data/higgs-data-1p.pickle"))
# X_train, X_test, y_train, y_test = pickle.load(open("../../higgs-data-1p.pickle"))
y_train = [y + y - 1 for y in y_train]
y_test = [y + y - 1 for y in y_test]

In [8]:
import pickle

(accuracy_train, auc_train), (accuracy_test, auc_test) = pickle.load(open("result/adaboost_trend.pkl"))
adaboost = pickle.load(open("result/adaboost.pkl"))

In [2]:
accuracy_train

[0.52976999999999996,
 0.60992000000000002,
 0.61260000000000003,
 0.61509999999999998,
 0.63485999999999998,
 0.65025999999999995,
 0.64932000000000001,
 0.65039999999999998,
 0.65719000000000005,
 0.65905000000000002,
 0.66134999999999999,
 0.65705000000000002,
 0.65919000000000005,
 0.66571999999999998,
 0.66571000000000002,
 0.66603999999999997,
 0.66627000000000003,
 0.66744000000000003,
 0.66781999999999997,
 0.67000000000000004,
 0.67213999999999996,
 0.67213000000000001,
 0.67188000000000003,
 0.67183000000000004,
 0.67256000000000005,
 0.67664000000000002,
 0.67681999999999998,
 0.67723,
 0.67906999999999995,
 0.68113000000000001,
 0.68140000000000001,
 0.68266000000000004,
 0.68430999999999997,
 0.68408999999999998,
 0.68408000000000002,
 0.68418000000000001,
 0.68694,
 0.68703999999999998,
 0.68706999999999996,
 0.68706,
 0.68730000000000002,
 0.68725999999999998,
 0.68725999999999998,
 0.68820999999999999,
 0.68825000000000003,
 0.68825000000000003,
 0.68825000000000003,
 0

In [12]:
for idx, k in enumerate(accuracy_train):
    if -1e-8 <= accuracy_train[idx + 1] - k <= 1e-8:
        print idx + 1
        break
accuracy_train[41], accuracy_train[42]

42


(0.68725999999999998, 0.68725999999999998)

```
=== Iteration 42 ===
Min score: 0.994264838088
Purity (farther from 1.0 is better): (1.0005403288644517, 1.0029535602499591)
Predicts (farther from 0.0 is better): (0.00027009146968671549, 0.0014746035301913612)
Split node: 10 (right)
Split index and value: 6 -1.54874491692 
```

In [25]:
node10 = adaboost[10]
insts = [(x, y) for (x, y) in zip(X_train, y_train) if node10.check(x, pre_check=False) == False]
len(insts)

1112

In [28]:
from numpy import exp

weights = []
for x, y in insts:
    score = 0.0
    for node in adaboost[:42]:
        score += node.predict(x, pre_check=False)
    weights.append(exp(-score * y))

In [46]:
from numpy import sqrt

node42 = adaboost[42]
left_pos, left_neg, right_pos, right_neg = [0.0] * 4
for idx, (x, y) in enumerate(insts):
    if node42.check(x) == True:
        if y == 1:
            left_pos += weights[idx]
        else:
            left_neg += weights[idx]
    else:
        if y == 1:
            right_pos += weights[idx]
        else:
            right_neg += weights[idx]
print left_pos, left_neg, right_pos, right_neg
print sqrt(left_pos * left_neg) + sqrt(right_pos * right_neg), left_pos + left_neg + right_pos + right_neg

31.0914786951 31.0746881441 363.616217593 362.545417858
394.16350526 788.32780229


In [44]:
node42.cond.index, node42.cond.val

(6, -1.5487449169158936)

In [48]:
node42 = adaboost[42]
vals = []
left_pos, left_neg, right_pos, right_neg = [0.0] * 4
for idx, (x, y) in enumerate(insts):
    if x[6] <= 0:
        if y == 1:
            left_pos += weights[idx]
        else:
            left_neg += weights[idx]
    else:
        if y == 1:
            right_pos += weights[idx]
        else:
            right_neg += weights[idx]
print left_pos, left_neg, right_pos, right_neg
print sqrt(left_pos * left_neg) + sqrt(right_pos * right_neg), left_pos + left_neg + right_pos + right_neg

191.507358611 198.386540508 203.200337677 195.233565494
394.093726864 788.32780229


# PROBLEM

Why the alternative split position has lower score in term of the loss function?

In [36]:
from numpy import log
from pyboost.utils import safe_comp

goto_cor, goto_err = 0, 0
cor, err = 0, 0
for idx, (x, y) in enumerate(insts):
    prev = log(weights[idx]) * -y
    score = prev + node42.predict(x)
    if safe_comp(score) != safe_comp(prev):
        if safe_comp(score) * y > 0:
            goto_cor += 1
        else:
            goto_err += 1
    if safe_comp(score) * y > 0:
        cor += 1
    else:
        err += 1
print goto_cor, goto_err
print cor, err

0 0
917 195


In [30]:
left_pos/left_neg

1.0005403288644517

In [31]:
right_pos/right_neg

1.0029535602499571